In [7]:
import numpy as np
import pandas as pd
import requests
from scipy import stats
import xlsxwriter

In [5]:
stocks = pd.read_csv('starter_files/sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

Making first api call

In [13]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2226199285355,
 'week52high': 149.53,
 'week52low': 77.32,
 'week52highSplitAdjustOnly': 146.44,
 'week52lowSplitAdjustOnly': 78.79,
 'week52change': 0.6754230704419859,
 'sharesOutstanding': 17222494549,
 'float': 0,
 'avg10Volume': 103770830,
 'avg30Volume': 95774157,
 'day200MovingAvg': 127.28,
 'day50MovingAvg': 131.53,
 'employees': 153909,
 'ttmEPS': 4.57,
 'ttmDividendRate': 0.8432751257465703,
 'dividendYield': 0.006703439587525757,
 'nextDividendDate': '0',
 'exDividendDate': '2021-05-03',
 'nextEarningsDate': '2021-07-25',
 'peRatio': 29.25654035210377,
 'beta': 1.4361134303364205,
 'maxChangePercent': 49.90477731401838,
 'year5ChangePercent': 5.177123520177282,
 'year2ChangePercent': 1.74284956481995,
 'year1ChangePercent': 0.6867642160688658,
 'ytdChangePercent': -0.03693967780920496,
 'month6ChangePercent': 0.07525019749490966,
 'month3ChangePercent': -0.057165312163051,
 'month1ChangePercent': -0.051811942622294,
 'day30ChangePer

In [14]:
data['year1ChangePercent']

0.6867642160688658

In [16]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))        
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One year Price Return', 'Number of share to buy']

In [23]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            
            [
                symbol,
                data[symbol]['price'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'
            ],
            index = my_columns),
            ignore_index=True
            
        ) 
        
final_dataframe

,Ticker,Price,One year Price Return,Number of share to buy
0,A,134.99,0.630436,N/A
1,AAL,22.50,1.49961,N/A
2,AAP,211.35,0.635032,N/A
3,AAPL,129.41,0.686959,N/A
4,ABBV,118.10,0.359581,N/A
...,...,...,...,...
500,YUM,123.17,0.452932,N/A
501,ZBH,169.32,0.514387,N/A
502,ZBRA,499.28,1.24676,N/A
503,ZION,62.70,1.34559,N/A


Remove Low-Momentum Stocks

In [32]:
final_dataframe.sort_values('One year Price Return', ascending=False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)

In [31]:
final_dataframe

,level_0,index,Ticker,Price,One year Price Return,Number of share to buy
0,0,275,LB,70.55,5.48929,N/A
1,1,179,FCX,43.91,4.18656,N/A
2,2,208,GPS,35.94,3.78819,N/A
3,3,253,IVZ,27.98,3.38862,N/A
4,4,441,TPR,48.63,2.92886,N/A
5,5,314,MOS,37.84,2.74934,N/A
6,6,272,KSS,63.86,2.51438,N/A
7,7,410,SIVB,594.39,2.37049,N/A
8,8,385,PWR,99.59,2.20766,N/A
9,9,129,DFS,119.88,2.20529,N/A
